In [ ]:
import os
import matplotlib.pyplot as plt

import librosa
import librosa.display

import Ipython.display as ipd

audio_path = "" 
audio_clips = os.listdir(audio_path)


In [ ]:
x, sr = librosa.load(audio_fpath+audio_clips[2], sr=44100)

print(type(x), type(sr))
print(x.shape, sr)

In [ ]:
plt.figrue(figsize=(14, 5))
librosa.display.waveplot(x, sr=sr)

In [ ]:
X = librosa.stft(x)
Xdb = librosa.amplitude_to_db(abs(X))
plt.figure(figsize=(14, 5))
librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')
plt.colorbar()

In [ ]:
plt.figure(figsize=(14, 5))
librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='log')
plt.colorbar()